In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load datasets
train_df = pd.read_csv(train_file_path, sep='\t', names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', names=['label', 'message'])


In [ ]:
# Data preprocessing
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Tokenization and Padding
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['message'])
X_train_seq = tokenizer.texts_to_sequences(train_df['message'])
X_test_seq = tokenizer.texts_to_sequences(test_df['message'])

X_train_pad = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_pad = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')

y_train = train_df['label'].values
y_test = test_df['label'].values


In [ ]:
# Model building
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=100),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Model training
history = model.fit(X_train_pad, y_train, epochs=10, validation_data=(X_test_pad, y_test), verbose=2)

In [ ]:
# Function to predict messages based on model
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded = keras.preprocessing.sequence.pad_sequences(seq, maxlen=100, padding='post', truncating='post')
    prediction = model.predict(padded)[0][0]
    label = 'spam' if prediction > 0.5 else 'ham'
    return [float(prediction), label]

In [ ]:
# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
